In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
def apply_gaussian_blur(img, sigma, kernel_size):
    device = img.device  # Store the original device
    img = img.squeeze(0).detach().cpu().numpy() #SHape is now 28x28
    img = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigmaX=sigma, sigmaY=sigma, borderType= cv2.BORDER_CONSTANT)
    img = torch.tensor(img, dtype=torch.float32).unsqueeze(0).to(device)  # Restore shape with correct device
    img.requires_grad = True
    return img

In [3]:
sigma1 = 1
kernel_size = 3
x = torch.zeros(kernel_size, kernel_size)
x[int(kernel_size/2), int(kernel_size/2)] = 1
x = x.type(torch.float32)
A_sigma1 = apply_gaussian_blur(x, sigma1, kernel_size)

In [13]:
A_sigma1 = A_sigma1.squeeze(0)

In [14]:
img_height = 28
img_width = 28
x = np.zeros((img_height, img_width))
total_dim = x.ndim

flat_size = 1
for i in x.shape:
  flat_size = flat_size * i

H = np.zeros((flat_size, flat_size))



In [24]:
for r in range(flat_size):
  for i in range(kernel_size):
    for j in range(kernel_size):
      n = r % img_height
      m = r // img_height

      if m - kernel_size//2 + i < 0 or m - kernel_size//2 + i >= img_height:
        continue
      if n - kernel_size//2 + j < 0 or n - kernel_size//2 + j >= img_width:
        continue

      H[(int)(m * img_width + n)
        ][(int)((m - kernel_size//2 + i) * img_width + (n - kernel_size//2 + j))
        ] = A_sigma1[i][j]

In [25]:
transform_clean = transforms.Compose([transforms.ToTensor()])
train_dataset = MNIST(root="./data", train=True, transform=transform_clean, download=True)
test_dataset = MNIST(root="./data", train=False, transform=transform_clean, download=True)

fraction = 1
num_samples = len(train_dataset)
indices = torch.arange(num_samples)[:int(fraction * num_samples)]

sampler = SubsetRandomSampler(indices)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, sampler = sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.97MB/s]


In [60]:
for clean_images, _ in train_loader:
  print(clean_images.shape)
  img = clean_images[0][0]
  y1 = apply_gaussian_blur(img, sigma=sigma1, kernel_size = kernel_size)
  y2 = H @ img.flatten().numpy()
  y2 = y2.reshape(28,28)
  y1 = y1[0].detach().numpy()
  ans = y1-y2
  break

torch.Size([32, 1, 28, 28])


In [77]:
torch.linalg.pinv(torch.tensor(H)) @ torch.tensor(H)

tensor([[ 1.0000e+00,  3.1444e-14, -2.1917e-14,  ...,  8.8818e-16,
         -2.1760e-14, -1.0880e-14],
        [ 7.9737e-16,  1.0000e+00,  7.8388e-15,  ..., -4.6629e-15,
          1.3101e-14,  5.1070e-15],
        [-1.8619e-15,  7.3525e-15,  1.0000e+00,  ...,  5.2180e-15,
         -5.6986e-16, -2.9057e-16],
        ...,
        [-2.3520e-15, -6.9388e-14,  1.0899e-13,  ...,  1.0000e+00,
          7.2831e-14,  2.4869e-14],
        [ 2.2674e-15,  5.4556e-14, -9.8697e-14,  ..., -1.9540e-14,
          1.0000e+00, -1.2434e-14],
        [-1.3303e-15, -3.5337e-14,  6.1706e-14,  ...,  7.1054e-15,
          3.6415e-14,  1.0000e+00]], dtype=torch.float64)